In [51]:
import torch.optim as optim
import numpy as np
import logger
from data_utils import *
#from model import SDEN
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score,accuracy_score,recall_score
import argparse

In [52]:
import torch

In [87]:
a = torch.tensor([[1,0,0,1,1]])

In [89]:
torch.tensor([torch.tensor(0),torch.tensor(0)])

tensor([0, 0])

In [86]:
sum([torch.sum(a),torch.sum(a)])

tensor(6)

In [69]:
torch.sum(a)

tensor(3)

In [80]:
a.type_as(torch.tensor(0.3))

tensor([[1., 0., 0., 1., 1.]])

In [8]:
path = '../data/m2m/train.iob'
print('building dictionary first...')
data = open(path,"r",encoding="utf-8").readlines()
p_data = []
bot = []
for d in data:
    if d=="\n":
        bot=[]
        continue
    dd = d.replace("\n","").split("|||")
    if len(dd)==1:
        bot = dd[0].split()
    else:
        user = dd[0].split()
        tag = dd[1].split()
        intent = dd[2]
        p_data.append([bot,user,tag,intent])
bots, currents, slots, intents = list(zip(*p_data))
# clean each word
vocab_freq = {}
for word in flatten(currents+bots):
    if vocab_freq.get(word)==None:
        vocab_freq[word]=1
    else:
        vocab_freq[word]+=1

building dictionary first...


In [19]:
sorted(vocab_freq.items(),key=lambda x:x[1])

[('giants', 1),
 ('undersand', 1),
 ('way', 1),
 ('shame', 1),
 ('bout', 1),
 ('home', 1),
 ('game', 1),
 ('hiya', 1),
 ('foods', 1),
 ('stadium', 1),
 ('quality', 1),
 ('end', 1),
 ('conversation', 1),
 ('shit', 1),
 ('nearest', 1),
 ('reserbvation', 1),
 ('concern', 1),
 ('pn', 1),
 ('alexander', 1),
 ('born', 1),
 ('fro', 1),
 ('google', 1),
 ('doesnt', 1),
 ('skip', 1),
 ('ciao', 1),
 ('fun', 1),
 ('retaurant', 1),
 ('esle', 1),
 ('country', 1),
 ('tasting', 1),
 ('coke', 1),
 ('gee', 1),
 ('hose', 1),
 ('forward', 1),
 ('suggestion', 1),
 ('hit', 1),
 ('brah', 1),
 ('obliged', 1),
 ('fore', 1),
 ('restraurant', 1),
 ('um', 1),
 ('our', 1),
 ('plan', 1),
 ('attendance', 1),
 ('anyhow', 1),
 ('soul', 1),
 ('old', 1),
 ('btw', 1),
 ('flexible', 1),
 ('nae', 1),
 ('important', 1),
 ('taking', 1),
 ('restervation', 1),
 ('birthday', 1),
 ('regardless', 1),
 ('actual', 1),
 ('job', 1),
 ('tallest', 1),
 ('tree', 1),
 ('world', 1),
 ('above', 1),
 ('provides', 1),
 ('feeling', 1),
 ('col

In [ ]:
from sklearn_crfsuite import metrics

In [21]:
labels = ['O','O','B-I']

In [22]:
sorted_labels = sorted(
        list(set(labels) - {'O', '<pad>'}),
        key=lambda name: (name[1:], name[0])
    )

In [23]:
sorted_labels

['B-I']

In [101]:
f1_score([1,0,0],[0,1,0],average='binary')

0.0

In [43]:
metrics.flat_classification_report(
        [['O'],['B-I'],['O']], [['O'],['O'],['B-I']], labels=['B-I'], digits=3).split('\n')[-2].split()[-2]

'0.000'

In [7]:
parser = argparse.ArgumentParser()
# DONOTCHANGE: They are reserved for nsml
parser.add_argument('--mode', type=str, default='train')
parser.add_argument('--pause', type=int, default=0)
parser.add_argument('--iteration', type=str, default='0')
parser.add_argument('--epochs', type=int, default=5,
                    help='num_epochs')
parser.add_argument('--batch_size', type=int, default=64,
                    help='batch size')
parser.add_argument('--lr', type=float, default=0.001,
                    help='learning_rate')
parser.add_argument('--dropout', type=float, default=0.3,
                    help='dropout')
parser.add_argument('--embed_size', type=int, default=100,
                    help='embed_size')
parser.add_argument('--hidden_size', type=int, default=64,
                    help='hidden_size')
parser.add_argument('--save_path', type=str, default='weight/model.pkl',
                    help='save_path')
parser.add_argument('--model', type=str, default='sden',
                    help='seq2seq, memory, sden' )
parser.add_argument('--slm',type=bool, default=True,
                    help='whether sentence level language model training or not')
parser.add_argument('--tensorboard',type=str, default='logs',
                    help='path for logs')
config = parser.parse_args(args=[])



In [8]:
train_data, train_slm_data, word2index, slot2index, intent2index = prepare_dataset('data/train.iob',slm=config.slm)
dev_data, dev_slm_data = prepare_dataset('data/dev.iob',(word2index,slot2index,intent2index),slm=config.slm)

if config.model == 'sden':
    model = SDEN(len(word2index),config.embed_size,config.hidden_size,\
                 len(slot2index),len(intent2index),word2index['<pad>'])

100%|██████████| 8793/8793 [00:01<00:00, 4649.02it/s]


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
model.vocab = word2index
model.slot_vocab = slot2index
model.intent_vocab = intent2index

log = logger.Logger(config.tensorboard)
